In [37]:
import numpy as np, sys

In [38]:
# Extract or Load data from mnist file
data = np.load("../Datasets/mnist.npz")

x_train = data["x_train"]
y_train = data["y_train"]
x_test = data["x_test"]
y_test = data["y_test"]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [39]:
# Take small set of data for quick training
x_train = x_train[0:1000]
labels = y_train[0:1000]
x_train = x_train.reshape(1000, 28 * 28)
images = x_train / 255

In [40]:
# one hot vectors for train
one_hot_vectors = np.zeros((len(labels), 10))
for idx, label in enumerate(labels):
    one_hot_vectors[idx][label] = 1
labels = one_hot_vectors

print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [41]:
# Testing dataset
test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
  test_labels[i][l] = 1
print(y_test)

[7 2 1 ... 4 5 6]


In [42]:
# Define functions


def tanh(x):
    return np.tanh(x)


def tanh2deriv(output):
    return 1 - (output**2)


def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

In [43]:
# Assign parameters
alpha, iterations = (0.1, 100)
pixels_per_image, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) * (input_cols - kernel_cols)) * num_kernels

kernels = np.random.random((kernel_rows * kernel_cols, num_kernels))

weights_1_2 = np.random.random((hidden_size, num_labels))

In [ ]:
def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:, row_from:row_to, col_from:col_to]
    return section.reshape(-1, 1, row_to - row_from, col_to - col_from)


for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i + 1) * batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(
                    layer_0,
                    row_start,
                    row_start + kernel_rows,
                    col_start,
                    col_start + kernel_cols,
                )
                sects.append(sect)

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_2 = softmax(np.dot(layer_1, weights_1_2))

        for k in range(batch_size):
            labelset = labels[batch_start + k : batch_start + k + 1]
            _inc = int(np.argmax(layer_2[k : k + 1]) == np.argmax(labelset))
            correct_cnt += _inc

        layer_2_delta = (labels[batch_start:batch_end] - layer_2) / (
            batch_size * layer_2.shape[0]
        )
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update

    test_correct_cnt = 0
    for i in range(len(test_images)):
        layer_0 = test_images[i : i + 1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(
                    layer_0,
                    row_start,
                    row_start + kernel_rows,
                    col_start,
                    col_start + kernel_cols,
                )
                sects.append(sect)

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        layer_2 = np.dot(layer_1, weights_1_2)

        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i : i + 1]))

        if j % 100 == 0:
            sys.stdout.write(
                "I:"
                + str(j)
                + " Test-Acc:"
                + str(test_correct_cnt / float(len(test_images)))
                + " Train-Acc:"
                + str(correct_cnt / float(len(images)))
                + "\n"
            )

C:\Users\Sdangol\AppData\Local\Temp\ipykernel_25312\495254846.py:13: RuntimeWarning: overflow encountered in exp
  temp = np.exp(x)
C:\Users\Sdangol\AppData\Local\Temp\ipykernel_25312\495254846.py:14: RuntimeWarning: invalid value encountered in divide
  return temp / np.sum(temp, axis=1, keepdims=True)


I:0 Test-Acc:0.0 Train-Acc:0.087
I:0 Test-Acc:0.0 Train-Acc:0.087
I:0 Test-Acc:0.0 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0001 Train-Acc:0.087
I:0 Test-Acc:0.0002 Train-Acc:0.087
I:0 Test-Acc:0.0002 Train-Acc:0.087
I:0 Test-Acc:0.0002 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0003 Train-Acc:0.087
I:0 Test-Acc:0.0004 Train-Acc:0.087
I:0 Test-Acc:0.0004 Train-Acc:0.087
I:0 Test-Acc:0.0004 Train-Acc:0.087
I